In [1]:
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

from tensorflow.keras.utils import to_categorical

from nltk.corpus import reuters

import os, string
import pandas as pd

import numpy as np

In [3]:
list_headlines = []

for word in reuters.sents():
    temp = ''
    temp = ' '.join([t for t in word if t])
    list_headlines.append(temp)

In [4]:
len(list_headlines)

54711

In [5]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in list_headlines]
corpus[:10]

['asian exporters fear damage from u  s  japan rift mounting trade friction between the u  s  and japan has raised fears among many of asia  s exporting nations that the row could inflict far  reaching economic damage  businessmen and officials said ',
 'they told reuter correspondents in asian capitals a u  s  move against japan might boost protectionist sentiment in the u  s  and lead to curbs on american imports of their products ',
 'but some exporters said that while the conflict would hurt them in the long  run  in the short  term tokyo  s loss might be their gain ',
 'the u  s  has said it will impose 300 mln dlrs of tariffs on imports of japanese electronics goods on april 17  in retaliation for japan  s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost ',
 'unofficial japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by t

In [6]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences

[[2139, 784],
 [2139, 784, 2567],
 [2139, 784, 2567, 1224],
 [2139, 784, 2567, 1224, 20],
 [2139, 784, 2567, 1224, 20, 29],
 [2139, 784, 2567, 1224, 20, 29, 9],
 [2139, 784, 2567, 1224, 20, 29, 9, 95],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282, 4635],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282, 4635, 63],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282, 4635, 63, 3820],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282, 4635, 63, 3820, 170],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282, 4635, 63, 3820, 170, 1],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282, 4635, 63, 3820, 170, 1, 29],
 [2139, 784, 2567, 1224, 20, 29, 9, 95, 13282, 4635, 63, 3820, 170, 1, 29, 9],
 [2139,
  784,
  2567,
  1224,
  20,
  29,
  9,
  95,
  13282,
  4635,
  63,
  3820,
  170,
  1,
  29,
  9,
  5],
 [2139,
  784,
  2567,
  1224,
  20,
  29,
  9,
  95,
  13282,
  4635,
  63,
  3820,
  170,
  1,
  29,
  9,
  5,
  95],
 [2139,
  784,
  2567,
  1224,
  20,
  29,
 

In [7]:
inp_sequences = inp_sequences[:25000]

In [8]:
len(inp_sequences)

25000

In [9]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [10]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 136, 10)           309520    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 30952)             3126152   
                                                                 
Total params: 3,480,072
Trainable params: 3,480,072
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
782/782 [==============================] - 75s 93ms/step - loss: 7.4439 - accuracy: 0.0396
Epoch 2/100
782/782 [==============================] - 77s 99ms/step - loss: 6.7116 - accuracy: 0.0473
Epoch 3/100
782/782 [==============================] - 85s 108ms/step - loss: 6.4250 - accuracy: 0.0662
Epoch 4/100
782/782 [==============================] - 64s 82ms/step - loss: 6.2102 - accuracy: 0.0811
Epoch 5/100
782/782 [==============================] - 61s 79ms/step - loss: 6.0329 - accuracy: 0.0942
Epoch 6/100
782/782 [==============================] - 69s 89ms/step - loss: 5.8577 - accuracy: 0.1030
Epoch 7/100
782/782 [==============================] - 69s 88ms/step - loss: 5.6956 - accuracy: 0.1122
Epoch 8/100
782/782 [==============================] - 60s 77ms/step - loss: 5.5340 - accuracy: 0.1178
Epoch 9/100
782/782 [==============================] - 68s 87ms/step - loss: 5.3756 - accuracy: 0.1246
Epoch 10/100
782/782 [==============================] - 72s 93ms/step - 

In [12]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted,axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [22]:
ins_kata = "the"
jml_kata = 10

print("Inisial Kata : ",ins_kata)
print("jumlah kata yang ingin digenerasi : ",jml_kata)
hasil_generasi = generate_text(ins_kata, jml_kata, model, max_sequence_len)
print("Hasil Generasi : ",hasil_generasi,"\n")

ins_kata = "the government"
jml_kata = 20

print("Inisial Kata : ",ins_kata)
print("jumlah kata yang ingin digenerasi : ",jml_kata)
hasil_generasi = generate_text(ins_kata, jml_kata, model, max_sequence_len)
print("Hasil Generasi : ",hasil_generasi,"\n")

ins_kata = "i like"
jml_kata = 5

print("Inisial Kata : ",ins_kata)
print("jumlah kata yang ingin digenerasi : ",jml_kata)
hasil_generasi = generate_text(ins_kata, jml_kata, model, max_sequence_len)
print("Hasil Generasi : ",hasil_generasi,"\n")

Inisial Kata :  the
jumlah kata yang ingin digenerasi :  10
Hasil Generasi :  The Company Said It Is Also Ready To Expand Effect Here 

Inisial Kata :  the government
jumlah kata yang ingin digenerasi :  20
Hasil Generasi :  The Government Agency Said That Harcourt Mining On International Debt With Trade U S And Lead To Curbs To Block Imports Of 

Inisial Kata :  i like
jumlah kata yang ingin digenerasi :  5
Hasil Generasi :  I Like Details And Sugar Reserves In 

